# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from langdetect import detect
from nltk.corpus import movie_reviews
from nltk.sentiment import SentimentIntensityAnalyzer
import random
from nltk.corpus import movie_reviews
from nltk.classify import apply_features

In [2]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin1', header=None)
df.columns = ['target','id','date','flag','user','text']

In [3]:
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
sample_df = df.sample(n=20000, random_state = 100)
sample_df = sample_df.reset_index(drop=True)
sample_df

,target,id,date,flag,user,text
0,4,1686122683,Sun May 03 03:52:46 PDT 2009,NO_QUERY,spunkgirl,@nicholasmw 1 day u will find that girl worry
1,0,2174460051,Sun Jun 14 22:35:53 PDT 2009,NO_QUERY,Bubbles_Fly,there is nothing on tv and im so desperate to ...
2,0,1795849965,Thu May 14 08:28:27 PDT 2009,NO_QUERY,ktfry246,Very excited that greys is on tonight. Not so ...
3,4,1933034865,Tue May 26 22:49:43 PDT 2009,NO_QUERY,Sheng07,2pm's again and again is a great song. Nichkhu...
4,0,1976523804,Sat May 30 18:21:19 PDT 2009,NO_QUERY,amberself,My teeth hurt
...,...,...,...,...,...,...
19995,4,1956795183,Thu May 28 22:42:33 PDT 2009,NO_QUERY,xdaniccca,@MRSmamaL I'm soooper excited!! I'm not going ...
19996,0,2203203675,Tue Jun 16 23:14:14 PDT 2009,NO_QUERY,StacySailer,Just watched the movie Wicked at Jimmy's... I ...
19997,0,2233834257,Thu Jun 18 21:58:44 PDT 2009,NO_QUERY,kieunga,I feel mad at dad right now cos suddenly i thi...
19998,0,2057777142,Sat Jun 06 13:38:29 PDT 2009,NO_QUERY,Shanz_x,"Watching big brother, not very good this year"


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# Functions from challenge 1
def clean_up(s):
    s = re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/?', ' ', s) # removing URL
    s = re.sub(r'[^\w\s]|\d|_', ' ', s) # removing special characters and numbers
    s = re.sub(' +', ' ', s) # removing extra spaces
    s = str(s).lower().strip()
    return s

def tokenize(s):
    tokens = word_tokenize(s)
    return tokens

def stem_and_lemmatize(l):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    text = [stemmer.stem(lemmatizer.lemmatize(i)) for i in l]
    return text

def remove_stopwords(l):
    stop_words = stopwords.words('english')
    tokens = [t for t in l if t not in stop_words]
    return tokens

In [6]:
from tqdm import tqdm
tqdm.pandas()

sample_df['text_processed'] = sample_df['text'].progress_apply(lambda x: clean_up(x))
sample_df['text_processed'] = sample_df['text_processed'].progress_apply(lambda x: tokenize(x))
sample_df['text_processed'] = sample_df['text_processed'].progress_apply(lambda x: stem_and_lemmatize(x))
sample_df['text_processed'] = sample_df['text_processed'].progress_apply(lambda x: remove_stopwords(x))

sample_df

100%|██████████| 20000/20000 [00:10<00:00, 1971.98it/s]


,target,id,date,flag,user,text,text_processed
0,4,1686122683,Sun May 03 03:52:46 PDT 2009,NO_QUERY,spunkgirl,@nicholasmw 1 day u will find that girl worry,"[nicholasmw, day, u, find, girl, worri]"
1,0,2174460051,Sun Jun 14 22:35:53 PDT 2009,NO_QUERY,Bubbles_Fly,there is nothing on tv and im so desperate to ...,"[noth, tv, im, desper, entertain, im, watch, h..."
2,0,1795849965,Thu May 14 08:28:27 PDT 2009,NO_QUERY,ktfry246,Very excited that greys is on tonight. Not so ...,"[veri, excit, grey, tonight, happi, season]"
3,4,1933034865,Tue May 26 22:49:43 PDT 2009,NO_QUERY,Sheng07,2pm's again and again is a great song. Nichkhu...,"[pm, great, song, nichkhun, hwait]"
4,0,1976523804,Sat May 30 18:21:19 PDT 2009,NO_QUERY,amberself,My teeth hurt,"[teeth, hurt]"
...,...,...,...,...,...,...,...
19995,4,1956795183,Thu May 28 22:42:33 PDT 2009,NO_QUERY,xdaniccca,@MRSmamaL I'm soooper excited!! I'm not going ...,"[mrsmamal, soooper, excit, go, ani, girl, nemo..."
19996,0,2203203675,Tue Jun 16 23:14:14 PDT 2009,NO_QUERY,StacySailer,Just watched the movie Wicked at Jimmy's... I ...,"[watch, movi, wick, jimmi, thought, wa, good, ..."
19997,0,2233834257,Thu Jun 18 21:58:44 PDT 2009,NO_QUERY,kieunga,I feel mad at dad right now cos suddenly i thi...,"[feel, mad, dad, right, co, suddenli, think, s..."
19998,0,2057777142,Sat Jun 06 13:38:29 PDT 2009,NO_QUERY,Shanz_x,"Watching big brother, not very good this year","[watch, big, brother, veri, good, thi, year]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [7]:
total_words = [word for row in sample_df['text_processed'] for word in row]

freq_dist = FreqDist(total_words)

top_words = freq_dist.most_common(5000)
top_words

[('go', 1743),
 ('get', 1429),
 ('day', 1371),
 ('wa', 1261),
 ('thi', 1198),
 ('good', 1115),
 ('work', 1091),
 ('love', 1029),
 ('like', 1003),
 ('quot', 938),
 ('got', 934),
 ('today', 874),
 ('u', 865),
 ('time', 854),
 ('want', 732),
 ('lol', 730),
 ('one', 717),
 ('back', 715),
 ('miss', 710),
 ('thank', 708),
 ('know', 693),
 ('see', 663),
 ('feel', 636),
 ('amp', 617),
 ('think', 614),
 ('im', 608),
 ('realli', 605),
 ('night', 590),
 ('watch', 577),
 ('still', 557),
 ('make', 544),
 ('new', 535),
 ('na', 533),
 ('oh', 512),
 ('need', 512),
 ('well', 506),
 ('look', 506),
 ('home', 499),
 ('come', 497),
 ('last', 492),
 ('hope', 491),
 ('ha', 475),
 ('much', 468),
 ('twitter', 455),
 ('morn', 452),
 ('sleep', 432),
 ('great', 427),
 ('tomorrow', 404),
 ('wish', 404),
 ('wait', 403),
 ('fun', 383),
 ('sad', 382),
 ('veri', 376),
 ('follow', 374),
 ('sorri', 372),
 ('tri', 362),
 ('onli', 359),
 ('haha', 357),
 ('week', 351),
 ('right', 343),
 ('whi', 336),
 ('friend', 336),
 ('h

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [8]:
bag_of_words = [word for word, _ in top_words]
num_documents = len(sample_df['text_processed'])
num_words = len(bag_of_words)
features = np.zeros((num_documents, num_words), dtype=bool)

for i, words_list in enumerate(sample_df['text_processed']):
    for word in words_list:
        if word in bag_of_words:
            word_index = bag_of_words.index(word)
            features[i, word_index] = True

feature_data = {word: features[:, i] for i, word in enumerate(bag_of_words)}
data = pd.DataFrame(feature_data)

In [9]:
feature_data

{'go': array([False, False, False, ..., False, False, False]),
 'get': array([False, False, False, ..., False, False, False]),
 'day': array([ True, False, False, ..., False, False, False]),
 'wa': array([False, False, False, ..., False, False, False]),
 'thi': array([False, False, False, ..., False,  True, False]),
 'good': array([False, False, False, ..., False,  True, False]),
 'work': array([False, False, False, ..., False, False, False]),
 'love': array([False, False, False, ..., False, False, False]),
 'like': array([False, False, False, ..., False, False, False]),
 'quot': array([False, False, False, ..., False, False, False]),
 'got': array([False, False, False, ..., False, False, False]),
 'today': array([False, False, False, ..., False, False, False]),
 'u': array([ True, False, False, ..., False, False, False]),
 'time': array([False, False, False, ..., False, False, False]),
 'want': array([False, False, False, ..., False, False, False]),
 'lol': array([False, False, False,

In [10]:
data

,go,get,day,wa,thi,good,work,love,like,quot,...,dvr,globe,stil,owwww,shadow,russel,admir,veget,protest,rochest
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19996,False,False,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19998,False,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    scores = sia.polarity_scores(text)
    compound_score = scores['compound']
    return compound_score > 0

# joining all words separated by spaces so we get a coherent string
data['positive_tone'] = data.apply(lambda row: get_sentiment(' '.join(row.index[row])), axis=1)
data

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,go,get,day,wa,thi,good,work,love,like,quot,...,globe,stil,owwww,shadow,russel,admir,veget,protest,rochest,positive_tone
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19996,False,False,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19998,False,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Building and Training Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [12]:
X = data.drop(columns=['positive_tone'])
y = data['positive_tone']

In [13]:
train_data = []
for i in range(len(X)):
    features = X.iloc[i].to_dict()
    label = y.iloc[i]
    train_data.append((features, label))

In [14]:
train_data[0]

({'go': False,
  'get': False,
  'day': True,
  'wa': False,
  'thi': False,
  'good': False,
  'work': False,
  'love': False,
  'like': False,
  'quot': False,
  'got': False,
  'today': False,
  'u': True,
  'time': False,
  'want': False,
  'lol': False,
  'one': False,
  'back': False,
  'miss': False,
  'thank': False,
  'know': False,
  'see': False,
  'feel': False,
  'amp': False,
  'think': False,
  'im': False,
  'realli': False,
  'night': False,
  'watch': False,
  'still': False,
  'make': False,
  'new': False,
  'na': False,
  'oh': False,
  'need': False,
  'well': False,
  'look': False,
  'home': False,
  'come': False,
  'last': False,
  'hope': False,
  'ha': False,
  'much': False,
  'twitter': False,
  'morn': False,
  'sleep': False,
  'great': False,
  'tomorrow': False,
  'wish': False,
  'wait': False,
  'fun': False,
  'sad': False,
  'veri': False,
  'follow': False,
  'sorri': False,
  'tri': False,
  'onli': False,
  'haha': False,
  'week': False,
  'rig

In [15]:
# Shuffling
random.shuffle(train_data)

# Train test split
train_size = int(0.8 * len(train_data))
train_set = train_data[:train_size]
test_set = train_data[train_size:]

# Training the model
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Show the most informative features
classifier.show_most_informative_features()

Most Informative Features
                   great = True             True : False  =     74.1 : 1.0
                    love = True             True : False  =     62.5 : 1.0
                    best = True             True : False  =     45.9 : 1.0
                     yay = True             True : False  =     42.0 : 1.0
                   smile = True             True : False  =     35.4 : 1.0
                    nice = True             True : False  =     30.6 : 1.0
                    luck = True             True : False  =     29.8 : 1.0
                   thank = True             True : False  =     28.3 : 1.0
                 perfect = True             True : False  =     28.0 : 1.0
                   bless = True             True : False  =     27.2 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [16]:
accuracy = nltk.classify.accuracy(classifier, test_set)
print("Accuracy:", accuracy)

Accuracy: 0.88325
